In [9]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.models.convnext import ConvNeXt_Base_Weights, ConvNeXt_Small_Weights
from torchvision.io import decode_image
from sklearn.model_selection import KFold
import numpy as np
import os
import cv2
from PIL import Image

In [10]:
# Prepare the data - create a KFold split
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# "Large Crowdcollected" dataset
authentic_path = "dane/CelebA/authentic"
spoof_path = "dane/CelebA/spoof"

# Load images
authentic_images = [os.path.join(authentic_path, img) for img in os.listdir(authentic_path)]
spoof_images = [os.path.join(spoof_path, img) for img in os.listdir(spoof_path)]

# Create X and y from all images
X = authentic_images + spoof_images
y = np.concatenate((np.zeros(len(authentic_images)), np.ones(len(spoof_images)))).astype(np.int64)

print("Total number of images: ", len(X))

# KFold
kf = KFold(n_splits=5, random_state=random_seed, shuffle=True)

Total number of images:  23052


In [11]:
torch.cuda.is_available()

False

In [12]:
# Custom dataset to load images and labels
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label, image_path  # Return image, label, and file path

In [13]:
# Create a transform to resize images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [14]:
def train_test_model(learning_rate=0.001, epochs=10, opt="adam"):
  # Create, test and evaluate the model for each fold
  for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
      print(f"Fold {fold}")
      X_train = [X[i] for i in train_idx]
      y_train = y[train_idx]
      X_test = [X[i] for i in test_idx]
      y_test = y[test_idx]

      # Create custom datasets
      train_dataset = OurDataset(X_train, y_train, transform=transform)
      test_dataset = OurDataset(X_test, y_test, transform=transform)

      # Create dataloaders
      train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
      test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

      # Device
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # Create a model
      model = models.convnext_small(weights=None)
      num_ftrs = model.classifier[-1].in_features
      model.classifier[-1] = nn.Linear(num_ftrs, 2)  # Two output neurons
      model = model.to(device)

      # Train the model
      criterion = nn.CrossEntropyLoss()

      if opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      elif opt == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

      num_epochs = epochs

      model.train()

      for epoch in range(num_epochs):
          model.train()
          running_loss = 0.0
          for images, labels, _ in train_loader:
              images, labels = images.to(device), labels.to(device)

              optimizer.zero_grad()
              outputs = model(images)
              loss = criterion(outputs, labels.long())
              loss.backward()
              optimizer.step()

              running_loss += loss.item()

          print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

      # Evaluate the model
      correct = 0
      total = 0
      incorrect_files = []

      accuracy = 0

      model.eval()

      with torch.no_grad():
          for images, labels, file_paths in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)

              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      print(f"Test Accuracy: {accuracy:.2f}%")
      print("-" * 50)

In [15]:
# Multiple paramter tests
# Baseline
train_test_model()

Fold 0
Epoch 1/10, Loss: 0.5864438571591104
Epoch 2/10, Loss: 0.35854603249444084
Epoch 3/10, Loss: 0.31969290207328993
Epoch 4/10, Loss: 0.28366725763993617
Epoch 5/10, Loss: 0.25892499080202197
Epoch 6/10, Loss: 0.2316239402085491
Epoch 7/10, Loss: 0.21876731819991327
Epoch 8/10, Loss: 0.20230947259936216
Epoch 9/10, Loss: 0.1922071899547269
Epoch 10/10, Loss: 0.17274001192469376
Test Accuracy: 92.58%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.6372571316655835
Epoch 2/10, Loss: 0.38061071729587725
Epoch 3/10, Loss: 0.33810267075767136
Epoch 4/10, Loss: 0.3191091822172988
Epoch 5/10, Loss: 0.2972848395231072
Epoch 6/10, Loss: 0.2825326896442078
Epoch 7/10, Loss: 0.2630932071218767
Epoch 8/10, Loss: 0.2392164236596387
Epoch 9/10, Loss: 0.22819416805055878
Epoch 10/10, Loss: 0.2047675254543278
Test Accuracy: 90.41%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.6140261815418611
Epoch 2/10, Loss: 0.3733409945328479
Epoch 3

In [16]:
# Increased LR
train_test_model(0.01)

Fold 0
Epoch 1/10, Loss: 0.7983547183628512
Epoch 2/10, Loss: 0.6883916986050598
Epoch 3/10, Loss: 0.6814512841829601
Epoch 4/10, Loss: 0.6792471783826421
Epoch 5/10, Loss: 0.6799139987864569
Epoch 6/10, Loss: 0.6793275787470866
Epoch 7/10, Loss: 0.6800407700150224
Epoch 8/10, Loss: 0.6790924751985837
Epoch 9/10, Loss: 0.6801657974203497
Epoch 10/10, Loss: 0.6805834939616185
Test Accuracy: 58.60%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.7801700186894712
Epoch 2/10, Loss: 0.6836370036548304
Epoch 3/10, Loss: 0.6799554977714499
Epoch 4/10, Loss: 0.6796147363644539
Epoch 5/10, Loss: 0.680453706035581
Epoch 6/10, Loss: 0.6800947570511539
Epoch 7/10, Loss: 0.6808035297245045
Epoch 8/10, Loss: 0.6802428457724573
Epoch 9/10, Loss: 0.6800805526548284
Epoch 10/10, Loss: 0.680546903011084
Test Accuracy: 59.01%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.73334390028304
Epoch 2/10, Loss: 0.6851956393937717
Epoch 3/10, Loss: 0.6

In [17]:
# Decreased LR, increased epochs
train_test_model(0.0001, 30)

Fold 0
Epoch 1/30, Loss: 0.43007881460353037
Epoch 2/30, Loss: 0.2957140548571144
Epoch 3/30, Loss: 0.2504778658354613
Epoch 4/30, Loss: 0.21901166120760363
Epoch 5/30, Loss: 0.18586185107094783
Epoch 6/30, Loss: 0.15653162922845762
Epoch 7/30, Loss: 0.12068680251917868
Epoch 8/30, Loss: 0.09194757218716386
Epoch 9/30, Loss: 0.07241181639972202
Epoch 10/30, Loss: 0.04729962874224298
Epoch 11/30, Loss: 0.037670198267544065
Epoch 12/30, Loss: 0.03205880672186476
Epoch 13/30, Loss: 0.031126524831058242
Epoch 14/30, Loss: 0.022559190163516662
Epoch 15/30, Loss: 0.017860284467421623
Epoch 16/30, Loss: 0.021622432950516194
Epoch 17/30, Loss: 0.016598132408271307
Epoch 18/30, Loss: 0.018086009946273407
Epoch 19/30, Loss: 0.018375413117036398
Epoch 20/30, Loss: 0.014984539952809307
Epoch 21/30, Loss: 0.012361413103584209
Epoch 22/30, Loss: 0.014039694245684034
Epoch 23/30, Loss: 0.012740945576041884
Epoch 24/30, Loss: 0.011630423913546243
Epoch 25/30, Loss: 0.013112073894089334
Epoch 26/30, Lo

In [18]:
# Change optimizer
train_test_model(0.001, 10, "sgd")

Fold 0
Epoch 1/10, Loss: 0.5967562254421426
Epoch 2/10, Loss: 0.4853337486137344
Epoch 3/10, Loss: 0.4662495965922519
Epoch 4/10, Loss: 0.45407667810313623
Epoch 5/10, Loss: 0.44523200638062826
Epoch 6/10, Loss: 0.43629185134864557
Epoch 7/10, Loss: 0.4287335377322119
Epoch 8/10, Loss: 0.42705530979294504
Epoch 9/10, Loss: 0.4207976467429662
Epoch 10/10, Loss: 0.4165204933600781
Test Accuracy: 80.68%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.6049203770929235
Epoch 2/10, Loss: 0.5043410478576217
Epoch 3/10, Loss: 0.4720627400811045
Epoch 4/10, Loss: 0.4632275973541898
Epoch 5/10, Loss: 0.4568854207285975
Epoch 6/10, Loss: 0.44833891393303665
Epoch 7/10, Loss: 0.4438922922264145
Epoch 8/10, Loss: 0.4372206488608074
Epoch 9/10, Loss: 0.43548960498238765
Epoch 10/10, Loss: 0.42900231377297504
Test Accuracy: 79.01%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.5961938899662507
Epoch 2/10, Loss: 0.4856754396259268
Epoch 3/10

In [19]:
# Change image size
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
train_test_model(0.001, 10)

Fold 0
Epoch 1/10, Loss: 0.5943166087776991
Epoch 2/10, Loss: 0.3678908957236251
Epoch 3/10, Loss: 0.3395348521953439
Epoch 4/10, Loss: 0.321097548708862
Epoch 5/10, Loss: 0.300084844722285
Epoch 6/10, Loss: 0.27461313230326106
Epoch 7/10, Loss: 0.2532311493572835
Epoch 8/10, Loss: 0.2287473608279662
Epoch 9/10, Loss: 0.2070001856435213
Epoch 10/10, Loss: 0.19325545796578947
Test Accuracy: 90.48%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.5101833377724297
Epoch 2/10, Loss: 0.3508998810650984
Epoch 3/10, Loss: 0.31617860598053843
Epoch 4/10, Loss: 0.27881224549648254
Epoch 5/10, Loss: 0.25989130715415065
Epoch 6/10, Loss: 0.22931057884901815
Epoch 7/10, Loss: 0.21226008201544164
Epoch 8/10, Loss: 0.1930755935512496
Epoch 9/10, Loss: 0.17715470910885822
Epoch 10/10, Loss: 0.16905495500195275
Test Accuracy: 91.89%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.6112960835521721
Epoch 2/10, Loss: 0.3683028061143034
Epoch 3/10

In [20]:
# Try pre-trained weights
def train_test_model(learning_rate=0.001, epochs=10, opt="adam"):
  # Create, test and evaluate the model for each fold
  for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
      print(f"Fold {fold}")
      X_train = [X[i] for i in train_idx]
      y_train = y[train_idx]
      X_test = [X[i] for i in test_idx]
      y_test = y[test_idx]

      # Create custom datasets
      train_dataset = OurDataset(X_train, y_train, transform=transform)
      test_dataset = OurDataset(X_test, y_test, transform=transform)

      # Create dataloaders
      train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
      test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

      # Device
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # Create a model
      model = models.convnext_small(weights=ConvNeXt_Small_Weights.DEFAULT)
      num_ftrs = model.classifier[-1].in_features
      model.classifier[-1] = nn.Linear(num_ftrs, 2)  # Two output neurons
      model = model.to(device)

      # Train the model
      criterion = nn.CrossEntropyLoss()

      if opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      elif opt == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

      num_epochs = epochs

      model.train()

      for epoch in range(num_epochs):
          model.train()
          running_loss = 0.0
          for images, labels, _ in train_loader:
              images, labels = images.to(device), labels.to(device)

              optimizer.zero_grad()
              outputs = model(images)
              loss = criterion(outputs, labels.long())
              loss.backward()
              optimizer.step()

              running_loss += loss.item()

          print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

      # Evaluate the model
      correct = 0
      total = 0
      incorrect_files = []

      accuracy = 0

      model.eval()

      with torch.no_grad():
          for images, labels, file_paths in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)

              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      print(f"Test Accuracy: {accuracy:.2f}%")
      print("-" * 50)
      
train_test_model(0.001, 10)

Fold 0


Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to C:\Users\rklinowski/.cache\torch\hub\checkpoints\convnext_small-0c510722.pth
100.0%


Epoch 1/10, Loss: 0.6889773135168713
Epoch 2/10, Loss: 0.6793568390494202
Epoch 3/10, Loss: 0.6790292339473705
Epoch 4/10, Loss: 0.678527961570634
Epoch 5/10, Loss: 0.6785082526801983
Epoch 6/10, Loss: 0.679001053243204
Epoch 7/10, Loss: 0.6791287828979294
Epoch 8/10, Loss: 0.6785836359243872
Epoch 9/10, Loss: 0.678320290724161
Epoch 10/10, Loss: 0.678273020625321
Test Accuracy: 58.60%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.690576872441343
Epoch 2/10, Loss: 0.6715376348697953
Epoch 3/10, Loss: 0.6790518473504528
Epoch 4/10, Loss: 0.678796140222037
Epoch 5/10, Loss: 0.6785699666269523
Epoch 6/10, Loss: 0.6790098796890761
Epoch 7/10, Loss: 0.679486604455852
Epoch 8/10, Loss: 0.6786374715213346
Epoch 9/10, Loss: 0.6790816998770993
Epoch 10/10, Loss: 0.6789359442076196
Test Accuracy: 59.01%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.6897943344231908
Epoch 2/10, Loss: 0.6794832910780684
Epoch 3/10, Loss: 0.67927780597

In [21]:
# Try larger model
# Try pre-trained weights
def train_test_model(learning_rate=0.001, epochs=10, opt="adam"):
  # Create, test and evaluate the model for each fold
  for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
      print(f"Fold {fold}")
      X_train = [X[i] for i in train_idx]
      y_train = y[train_idx]
      X_test = [X[i] for i in test_idx]
      y_test = y[test_idx]

      # Create custom datasets
      train_dataset = OurDataset(X_train, y_train, transform=transform)
      test_dataset = OurDataset(X_test, y_test, transform=transform)

      # Create dataloaders
      train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
      test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

      # Device
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # Create a model
      model = models.convnext_base(weights=ConvNeXt_Base_Weights.DEFAULT)
      num_ftrs = model.classifier[-1].in_features
      model.classifier[-1] = nn.Linear(num_ftrs, 2)  # Two output neurons
      model = model.to(device)

      # Train the model
      criterion = nn.CrossEntropyLoss()

      if opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      elif opt == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

      num_epochs = epochs

      model.train()

      for epoch in range(num_epochs):
          model.train()
          running_loss = 0.0
          for images, labels, _ in train_loader:
              images, labels = images.to(device), labels.to(device)

              optimizer.zero_grad()
              outputs = model(images)
              loss = criterion(outputs, labels.long())
              loss.backward()
              optimizer.step()

              running_loss += loss.item()

          print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

      # Evaluate the model
      correct = 0
      total = 0
      incorrect_files = []

      accuracy = 0

      model.eval()

      with torch.no_grad():
          for images, labels, file_paths in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)

              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      print(f"Test Accuracy: {accuracy:.2f}%")
      print("-" * 50)
      
train_test_model(0.001, 10)

Fold 0


Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to C:\Users\rklinowski/.cache\torch\hub\checkpoints\convnext_base-6075fbad.pth
100.0%


Epoch 1/10, Loss: 0.683315446099954
Epoch 2/10, Loss: 0.6799539015652608
Epoch 3/10, Loss: 0.6785225444691226
Epoch 4/10, Loss: 0.6788001718512854
Epoch 5/10, Loss: 0.6783458237218278
Epoch 6/10, Loss: 0.6784349937166251
Epoch 7/10, Loss: 0.6810457049457411
Epoch 8/10, Loss: 0.6784382879837341
Epoch 9/10, Loss: 0.6787161734116552
Epoch 10/10, Loss: 0.6784609323770996
Test Accuracy: 58.60%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.6823479689179921
Epoch 2/10, Loss: 0.6792367583131046
Epoch 3/10, Loss: 0.6790203209353072
Epoch 4/10, Loss: 0.6793005780496052
Epoch 5/10, Loss: 0.6792366592474992
Epoch 6/10, Loss: 0.6791103381011474
Epoch 7/10, Loss: 0.6791604217045022
Epoch 8/10, Loss: 0.6786704772060004
Epoch 9/10, Loss: 0.678723426377546
Epoch 10/10, Loss: 0.6786904148047264
Test Accuracy: 59.01%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.6871975216130037
Epoch 2/10, Loss: 0.680417519095039
Epoch 3/10, Loss: 0.6798911